## TP Final - Optimizacion

In [1]:
import cplex

In [2]:
TOLERANCE =10e-6 

class Orden:
    def __init__(self):
        self.id = 0
        self.beneficio = 0
        self.trabajadores_necesarios = 0
    
    def load(self, row):
        self.id = int(row[0])
        self.beneficio = int(row[1])
        self.trabajadores_necesarios = int(row[2])
        

class FieldWorkAssignment:
    def __init__(self):
        self.cantidad_trabajadores = 0
        self.cantidad_ordenes = 0
        self.ordenes = []
        self.conflictos_trabajadores = []
        self.ordenes_correlativas = [] ##S
        self.ordenes_conflictivas = [] ##C
        self.ordenes_repetitivas = []
        self.diccionario_indices = {}
        self.variables_totales=0
        self.tramos_salario=4
        self.valor_tramo=[-1_000, -1_200,-1_400,-1_500 ]
        self.dias_semana=6
        self.turnos=5
        self.variables=[]
        self.variables_binarias=[]
        self.variables_N=[]
        

    def load(self,filename):
        # Abrimos el archivo.
        f = open(filename)

        # Leemos la cantidad de trabajadores
        self.cantidad_trabajadores = int(f.readline())
        
        # Leemos la cantidad de ordenes
        self.cantidad_ordenes = int(f.readline())
        
        # Leemos cada una de las ordenes.
        self.ordenes = []
        for i in range(self.cantidad_ordenes):
            row = f.readline().split(' ')
            orden = Orden()
            orden.load(row)
            self.ordenes.append(orden)

        # Leemos la cantidad de conflictos entre los trabajadores
        cantidad_conflictos_trabajadores = int(f.readline())
        
        # Leemos los conflictos entre los trabajadores
        self.conflictos_trabajadores = []
        for i in range(cantidad_conflictos_trabajadores):
            row = f.readline().split(' ')
            self.conflictos_trabajadores.append(list(map(int,row)))
            
        # Leemos la cantidad de ordenes correlativas
        cantidad_ordenes_correlativas = int(f.readline())
        
        # Leemos las ordenes correlativas
        self.ordenes_correlativas = []
        for i in range(cantidad_ordenes_correlativas):
            row = f.readline().split(' ')
            self.ordenes_correlativas.append(list(map(int,row)))
            
        # Leemos la cantidad de ordenes conflictivas
        cantidad_ordenes_conflictivas = int(f.readline())
        
        # Leemos las ordenes conflictivas
        self.ordenes_conflictivas = []
        for i in range(cantidad_ordenes_conflictivas):
            row = f.readline().split(' ')
            self.ordenes_conflictivas.append(list(map(int,row)))
        
        
        # Leemos la cantidad de ordenes repetitivas
        cantidad_ordenes_repetitivas = int(f.readline())
        
        # Leemos las ordenes repetitivas
        self.ordenes_repetitivas = []
        for i in range(cantidad_ordenes_repetitivas):
            row = f.readline().split(' ')
            self.ordenes_repetitivas.append(list(map(int,row)))


        
        I=self.cantidad_trabajadores  #trabajador
        J=self.cantidad_ordenes  #orden o tarea
        K=self.dias_semana  #dia de semana
        L=self.turnos  #turno
        N=self.tramos_salario  #tramos
        W=N-1  #auxiliar activa tramos
        for i in range(I):  # i 
            for j in range(J):  # j 
                for k in range(K):  # k 
                    for l in range(L):  # l 
                        clave = f"X_{i}_{j}_{k}_{l}"  
                        valor = i * L*K*J + j * L*K + k * L + l  # Valor correlativo
                        self.diccionario_indices[clave] = valor

        max_valor = max(self.diccionario_indices.values())
        for i in range(I):  # i 
            for k in range(K):  # k 
                clave = f"D_{i}_{k}"
                valor = max_valor+1+i*K+k
                self.diccionario_indices[clave] = valor

        max_valor = max(self.diccionario_indices.values())
        for j in range(J):  # j 
            for k in range(K):  # k 
                for l in range(L):  # l 
                    clave = f"R_{j}_{k}_{l}"  
                    valor = max_valor+1+ j * L*K + k * L + l  # Valor correlativo
                    self.diccionario_indices[clave] = valor

        max_valor = max(self.diccionario_indices.values())
        for i in range(I):  # i 
            for w in range(W):  # n  
                clave = f"W_{i}_{w}"
                valor = max_valor+1+i*W+w
                self.diccionario_indices[clave] = valor
                
        max_valor = max(self.diccionario_indices.values())
        for i in range(I):  # i 
            for n in range(N):  # n  
                clave = f"N_{i}_{n}"
                valor = max_valor+1+i*N+n
                self.diccionario_indices[clave] = valor
        max_valor = max(self.diccionario_indices.values())

        self.variables=[0]* len(self.diccionario_indices)
        # Cerramos el archivo.
        f.close()

       

def get_instance_data():
    #file_location = sys.argv[1].strip()
    file_location='data/input_data_P2_5.txt'
    instance = FieldWorkAssignment()
    instance.load(file_location)
    return instance
     


In [3]:
def genera_condiciones_variables():
    
    for key in data.diccionario_indices.keys():
        if key[0]=='R':
            for j in range(data.cantidad_ordenes):
                if key[2]==str(j):
                    posicion=data.diccionario_indices[key]
                    data.variables[posicion]=data.ordenes[j].beneficio
    for key in data.diccionario_indices.keys():
        if key[0]=='N':
            for numero_tramo in range(data.tramos_salario):
                if key[4]==str(numero_tramo):
                    posicion=data.diccionario_indices[key]
                    data.variables[posicion]=data.valor_tramo[numero_tramo]
    data.variables_N=data.variables[data.diccionario_indices['N_0_0']:]
    data.variables_binarias=data.variables[:data.diccionario_indices['N_0_0']]

In [4]:
# Obtenemos los datos de la instancia.
data = get_instance_data()
genera_condiciones_variables()
# Definimos el problema de cplex.
my_problem = cplex.Cplex()




print(list(data.diccionario_indices.keys())[5])
print(list(data.diccionario_indices.keys())[5].find('_'))
print(list(data.diccionario_indices.keys())[5].rfind('_'))
ver_una_posicion=3333
print(list(data.diccionario_indices.keys())[ver_una_posicion])
print([i for i, char in enumerate(list(data.diccionario_indices.keys())[ver_una_posicion]) if char == '_'])
t='R_1_21_3'
#t='R_19_0_3'
print(t)
print([posicion for posicion, char in enumerate(t) if char == '_'])
p=1
posiciones=[posicion for posicion, char in enumerate(t) if char == '_']
print(posiciones)
t[posiciones[p]+1:posiciones[p+1]]


In [5]:
def add_constraint_matrix_1(my_problem, data):
# R_k_l  <= a 1 para cada j  R_{j}_{k}_{l}
    for j in range(data.cantidad_ordenes):
        restriccion_indices=[]
        restriccion_valores=[]
        for t in data.diccionario_indices.keys():
            
            posiciones=[posicion for posicion, char in enumerate(t) if char == '_']
            if t[0]=='R'and t[posiciones[0]+1:posiciones[0+1]]==str(j):
            #if t[0]=='R'and t[2]==str(j):
            
                
                restriccion_indices.append(data.diccionario_indices[t])
                restriccion_valores.append(1)
                
        indices = restriccion_indices
        values = restriccion_valores
        row = [indices,values]
        my_problem.linear_constraints.add(lin_expr=[row], senses=['L'], rhs=[1])               

In [6]:
def add_constraint_matrix_2(my_problem, data):
# D-Sum(X_j_l)<=0 para todo i k   D_{i}_{k} ||X_{i}_{j}_{k}_{l}
        for i in range(data.cantidad_trabajadores):
                for k in range(data.dias_semana):        
                        restriccion_indices=[]
                        restriccion_valores=[]
                        for t in data.diccionario_indices.keys():
                                posiciones=[posicion for posicion, char in enumerate(t) if char == '_']
                                #cuando es el ultimo no termina en la posicion siguiente sino en el final
                                if t[0]=='D'and t[posiciones[0]+1:posiciones[0+1]]==str(i) and t[posiciones[1]+1:]==str(k):
                                #if t[0]=='D'and t[2]==str(i) and t[4]==str(k):
                                        restriccion_indices.append(data.diccionario_indices[t])
                                        restriccion_valores.append(1)
                                elif t[0]=='X'and t[posiciones[0]+1:posiciones[0+1]]==str(i) and t[posiciones[2]+1:posiciones[2+1]]==str(k):
                                #elif t[0]=='X'and t[2]==str(i) and t[6]==str(k):
                                        restriccion_indices.append(data.diccionario_indices[t])
                                        restriccion_valores.append(-1)                                                
                        indices = restriccion_indices
                        values = restriccion_valores
                        row = [indices,values]
                        my_problem.linear_constraints.add(lin_expr=[row], senses=['L'], rhs=[0])         


In [7]:
def add_constraint_matrix_3(my_problem, data):
#X_{i}_{j}_{k}_{l}
#   0   1   2   3:
#Sum(X_j)<=1 para todo i,k,l
        for i in range(data.cantidad_trabajadores):
                for k in range(data.dias_semana):
                        for l in range(data.turnos):
                                restriccion_indices=[]
                                restriccion_valores=[]
                                for t in data.diccionario_indices.keys():
                                        posiciones=[posicion for posicion, char in enumerate(t) if char == '_']
                                        if t[0]=='X'and t[posiciones[0]+1:posiciones[0+1]]==str(i) and t[posiciones[2]+1:posiciones[2+1]]==str(k) and t[posiciones[3]+1:]==str(l):        
                                        #if t[0]=='X'and t[2]==str(i) and t[6]==str(k) and t[8]==str(l):
                                                restriccion_indices.append(data.diccionario_indices[t])
                                                restriccion_valores.append(1)
                                
                                indices = restriccion_indices
                                values = restriccion_valores
                                row = [indices,values]
                                if len(indices)>0:
                                        my_problem.linear_constraints.add(lin_expr=[row], senses=['L'], rhs=[1])
                                else: 
                                        break
                        #                print(restriccion_indices)
                        #                print(restriccion_valores)
                        #                print(len(restriccion_valores))

In [8]:
def add_constraint_matrix_4(my_problem, data):
#X_{i}_{j}_{k}_{l}
#   0   1   2   3:
#Sum(X_j_l)    <=4 para todo i,k
        for i in range(data.cantidad_trabajadores):
                for k in range(data.dias_semana):
                        
                        restriccion_indices=[]
                        restriccion_valores=[]
                        for t in data.diccionario_indices.keys():
                                posiciones=[posicion for posicion, char in enumerate(t) if char == '_']
                                if t[0]=='X'and t[posiciones[0]+1:posiciones[0+1]]==str(i) and t[posiciones[2]+1:posiciones[2+1]]==str(k):
                                #if t[0]=='X'and t[2]==str(i) and t[6]==str(k):

                                        restriccion_indices.append(data.diccionario_indices[t])
                                        restriccion_valores.append(1)
                        
                        indices = restriccion_indices
                        values = restriccion_valores
                        row = [indices,values]
                        if len(indices)>0:
                                my_problem.linear_constraints.add(lin_expr=[row], senses=['L'], rhs=[4])
                        else: 
                                break

In [9]:
def add_constraint_matrix_5(my_problem, data):
#D_{i}_{k} ||X_{i}_{j}_{k}_{l}    
# D_k    <=5 para todo i
        for i in range(data.cantidad_trabajadores):
                restriccion_indices=[]
                restriccion_valores=[]
                for t in data.diccionario_indices.keys():
                        posiciones=[posicion for posicion, char in enumerate(t) if char == '_']
                        if t[0]=='D'and t[posiciones[0]+1:posiciones[0+1]]==str(i):        
                        #if t[0]=='D'and t[2]==str(i):
                                restriccion_indices.append(data.diccionario_indices[t])
                                restriccion_valores.append(1)
                                             
                indices = restriccion_indices
                values = restriccion_valores
                row = [indices,values]
                my_problem.linear_constraints.add(lin_expr=[row], senses=['L'], rhs=[5]) 
                        #                print(restriccion_indices)
                        #                print(restriccion_valores)
                        #                print(len(restriccion_valores))

In [10]:
def add_constraint_matrix_6(my_problem, data):
#X_{i}_{j}_{k}_{l}  ||  R_{j}_{k}_{l}   ||D_{i}_{k} 
#Sum(X_i) -T* R =0 para todo j,k,l
        for j in range(data.cantidad_ordenes):
                for k in range(data.dias_semana):
                        for l in range(data.turnos):
                                restriccion_indices=[]
                                restriccion_valores=[]
                                for t in data.diccionario_indices.keys():
                                        posiciones=[posicion for posicion, char in enumerate(t) if char == '_']
                                        if t[0]=='X'and t[posiciones[1]+1:posiciones[1+1]]==str(j) and t[posiciones[2]+1:posiciones[2+1]]==str(k) and t[posiciones[3]+1:]==str(l):
                                        #if t[0]=='X'and t[4]==str(j) and t[6]==str(k) and t[8]==str(l):
                                                restriccion_indices.append(data.diccionario_indices[t])
                                                restriccion_valores.append(1)
                                        elif t[0]=='R'and t[posiciones[0]+1:posiciones[0+1]]==str(j) and t[posiciones[1]+1:posiciones[1+1]]==str(k) and t[posiciones[2]+1:]==str(l):
                                        #elif t[0]=='R'and t[2]==str(j) and t[4]==str(k) and t[6]==str(l):
                                                restriccion_indices.append(data.diccionario_indices[t])
                                                restriccion_valores.append(-1*(data.ordenes[j].trabajadores_necesarios)) #T cantidad de trabajadores por tarea                       
                                indices = restriccion_indices
                                values = restriccion_valores
                                row = [indices,values]
                                if len(indices)>0:
                                        my_problem.linear_constraints.add(lin_expr=[row], senses=['E'], rhs=[0])
                                else: 
                                        break
                #                print(restriccion_indices)
                #                print(restriccion_valores)
                #                print(len(restriccion_valores))
                #data.ordenes[k].trabajadores_necesarios

In [11]:
def add_constraint_matrix_7A(my_problem, data): # la restriccion respeta el orden de la info en el archivo
#X_{i}_{j}_{k}_{l}                  #R_{j}_{k}_{l}
#R_j1__k_l +R_j2__k_l+1 - C_j1_j2=1 
#data.ordenes_conflictivas lista [[1, 3], [2, 5], [3, 6]]
### 1*R_1_0_0 +1*R_3_0_1=1  ||    1*R_1_0_1 +1*R_3_0_2=1   ||  1*R_1_1_0 +1*R_3_1_1....=1
### 1*X_0_1_0_0 +1*X_0_3_0_1+  1*X_0_1_1_0 +1*X_0_3_2_1    =1  ||
        for k in range(data.dias_semana):
                for l in range(data.turnos-1):
                        for i in range(data.cantidad_trabajadores): 
                                for j1, j2 in data.ordenes_conflictivas:  #listado de conflictivas que provienen de la data
                                        restriccion_indices=[]
                                        restriccion_valores=[]       
                                        for t in data.diccionario_indices.keys():
                                                posiciones=[posicion for posicion, char in enumerate(t) if char == '_']
                                                
                                                if t[0]=='X'and t[posiciones[0]+1:posiciones[0+1]]==str(i) and t[posiciones[1]+1:posiciones[1+1]]==str(j1) and t[posiciones[2]+1:posiciones[2+1]]==str(k) and t[posiciones[3]+1:]==str(l):
                                                        restriccion_indices.append(data.diccionario_indices[t])
                                                        restriccion_valores.append(1)
                                                elif t[0]=='X'and t[posiciones[0]+1:posiciones[0+1]]==str(i) and t[posiciones[1]+1:posiciones[1+1]]==str(j2) and t[posiciones[2]+1:posiciones[2+1]]==str(k) and t[posiciones[3]+1:]==str(l+1):
                                                        restriccion_indices.append(data.diccionario_indices[t])
                                                        restriccion_valores.append(1)                       
                                        indices = restriccion_indices
                                        values = restriccion_valores
                                        row = [indices,values]
                                        
                                        if len(indices)>0:
                                                my_problem.linear_constraints.add(lin_expr=[row], senses=['L'], rhs=[1])
                                        else: 
                                                break
                #                print(restriccion_indices)
                #                print(restriccion_valores)
                #                print(len(restriccion_valores))
#data.ordenes[k].trabajadores_necesarios
#print(row)
#Validacion ultima restriccion
#data.diccionario_indices['X_9_3_5_3']
#data.diccionario_indices['X_9_6_5_4']
#row

In [12]:
def add_constraint_matrix_7B(my_problem, data): # la restriccion altera el orden de la info en el archivo J1 potr J2
        for k in range(data.dias_semana):
                for l in range(data.turnos-1):
                        for i in range(data.cantidad_trabajadores): 
                                for j1, j2 in data.ordenes_conflictivas:  #listado de conflictivas que provienen de la data
                                        restriccion_indices=[]
                                        restriccion_valores=[]       
                                        for t in data.diccionario_indices.keys():
                                                posiciones=[posicion for posicion, char in enumerate(t) if char == '_']
                                                
                                                if t[0]=='X'and t[posiciones[0]+1:posiciones[0+1]]==str(i) and t[posiciones[1]+1:posiciones[1+1]]==str(j2) and t[posiciones[2]+1:posiciones[2+1]]==str(k) and t[posiciones[3]+1:]==str(l):
                                                        restriccion_indices.append(data.diccionario_indices[t])
                                                        restriccion_valores.append(1)
                                                elif t[0]=='X'and t[posiciones[0]+1:posiciones[0+1]]==str(i) and t[posiciones[1]+1:posiciones[1+1]]==str(j1) and t[posiciones[2]+1:posiciones[2+1]]==str(k) and t[posiciones[3]+1:]==str(l+1):
                                                        restriccion_indices.append(data.diccionario_indices[t])
                                                        restriccion_valores.append(1)                       
                                        indices = restriccion_indices
                                        values = restriccion_valores
                                        row = [indices,values]
                                        
                                        if len(indices)>0:
                                                my_problem.linear_constraints.add(lin_expr=[row], senses=['L'], rhs=[1])
                                        else: 
                                                break

In [13]:
def add_constraint_matrix_8(my_problem, data):
# X_i_j_k_l - D_i_k <=0 para todo i,j,k,l     #X_{i}_{j}_{k}_{l}  ||  R_{j}_{k}_{l}   ||D_{i}_{k} 
        for i in range(data.cantidad_trabajadores):
                for k in range(data.dias_semana):        
                        for l in range(data.turnos):
                                for j in range(data.cantidad_ordenes):
                                        restriccion_indices=[]
                                        restriccion_valores=[]
                                        for t in data.diccionario_indices.keys():
                                                posiciones=[posicion for posicion, char in enumerate(t) if char == '_']
                                                if t[0]=='D'and t[posiciones[0]+1:posiciones[0+1]]==str(i) and t[posiciones[1]+1:]==str(k):
                                                        restriccion_indices.append(data.diccionario_indices[t])
                                                        restriccion_valores.append(-1)
                                                elif t[0]=='X'and t[posiciones[0]+1:posiciones[0+1]]==str(i) and t[posiciones[1]+1:posiciones[1+1]]==str(j) and t[posiciones[2]+1:posiciones[2+1]]==str(k) and t[posiciones[3]+1:]==str(l):
                                                        restriccion_indices.append(data.diccionario_indices[t])
                                                        restriccion_valores.append(1)                                                
                                        indices = restriccion_indices
                                        values = restriccion_valores 
                                        row = [indices,values]
                                        my_problem.linear_constraints.add(lin_expr=[row], senses=['L'], rhs=[0])         


In [14]:
def add_constraint_matrix_9(my_problem, data): # Reestriccion de Correlativas
#  X_{i}_{j}_{k}_{l}  ||  R_{j}_{k}_{l}   ||D_{i}_{k} 
# S_j1_j2 * ( R_j1__k_l  - R_j2__k_l+1)  = 0   
#data.ordenes_correlativas lista [[1, 2]]  Solo se activa cuando esta en la lista de correlativas

        for k in range(data.dias_semana):
                for l in range(data.turnos-1):
                        for j1, j2 in data.ordenes_correlativas:  #listado de correlativas que provienen de la data
                                restriccion_indices=[]
                                restriccion_valores=[]       
                                for t in data.diccionario_indices.keys():
                                        posiciones=[posicion for posicion, char in enumerate(t) if char == '_']
                                        
                                        if t[0]=='R'and t[posiciones[0]+1:posiciones[0+1]]==str(j1)  and t[posiciones[1]+1:posiciones[1+1]]==str(k) and t[posiciones[2]+1:]==str(l):
                                                restriccion_indices.append(data.diccionario_indices[t])
                                                restriccion_valores.append(1)
                                        elif t[0]=='R'and t[posiciones[0]+1:posiciones[0+1]]==str(j2) and t[posiciones[1]+1:posiciones[1+1]]==str(k) and t[posiciones[2]+1:]==str(l+1):
                                                restriccion_indices.append(data.diccionario_indices[t])
                                                restriccion_valores.append(-1)                       
                                indices = restriccion_indices
                                values = restriccion_valores
                                row = [indices,values]
      
                                if len(indices)>0:
                                        my_problem.linear_constraints.add(lin_expr=[row], senses=['E'], rhs=[0])
                                else: 
                                        break

In [15]:
def add_constraint_matrix_10(my_problem, data):
# SUM_jkl (X_i1_j_k_l) -SUM_jkl X_i2_j_k_l <= 10 para todo i1<>i2      #X_{i}_{j}_{k}_{l}  ||  R_{j}_{k}_{l}   ||D_{i}_{k} 
        for i1 in range(data.cantidad_trabajadores):
                for i2 in range(data.cantidad_trabajadores):
                        if i1!=i2:                
                                restriccion_indices=[]
                                restriccion_valores=[]
                                for t in data.diccionario_indices.keys():
                                        posiciones=[posicion for posicion, char in enumerate(t) if char == '_']
                                        if t[0]=='X'and t[posiciones[0]+1:posiciones[0+1]]==str(i1):
                                                restriccion_indices.append(data.diccionario_indices[t])
                                                restriccion_valores.append(1)
                                        elif t[0]=='X'and t[posiciones[0]+1:posiciones[0+1]]==str(i2):
                                                restriccion_indices.append(data.diccionario_indices[t])
                                                restriccion_valores.append(-1)                                                
                                indices = restriccion_indices
                                values = restriccion_valores
                                row = [indices,values]
                                
                                if len(indices)>0:
                                        my_problem.linear_constraints.add(lin_expr=[row], senses=['L'], rhs=[10])
                                else: 
                                        break   

In [16]:
def add_constraint_matrix_11(my_problem, data):
# SUM_n (N_i_n) -SUM_jkl (X_i_j_k_l) = 0 para todo i      #X_{i}_{j}_{k}_{l}  ||  R_{j}_{k}_{l}   ||D_{i}_{k} || N_{i}_{n} || W_{i}_{w}
        for i in range(data.cantidad_trabajadores):
                restriccion_indices=[]
                restriccion_valores=[]
                for t in data.diccionario_indices.keys():
                        posiciones=[posicion for posicion, char in enumerate(t) if char == '_']
                        if t[0]=='N'and t[posiciones[0]+1:posiciones[0+1]]==str(i):
                                restriccion_indices.append(data.diccionario_indices[t])
                                restriccion_valores.append(1)
                        elif t[0]=='X'and t[posiciones[0]+1:posiciones[0+1]]==str(i):
                                restriccion_indices.append(data.diccionario_indices[t])
                                restriccion_valores.append(-1)                                                
                indices = restriccion_indices
                values = restriccion_valores
                row = [indices,values]
                if len(indices)>0:
                        my_problem.linear_constraints.add(lin_expr=[row], senses=['E'], rhs=[0])
                else: 
                        break      

In [17]:
def add_constraint_matrix_12(my_problem, data):
# 5 * W_i_0 - N_i_0 = 0 para todo i  para     #X_{i}_{j}_{k}_{l}  ||  R_{j}_{k}_{l}   ||D_{i}_{k} || N_{i}_{n} || W_{i}_{w}
        for i in range(data.cantidad_trabajadores):
                restriccion_indices=[]
                restriccion_valores=[]
                for t in data.diccionario_indices.keys():
                        posiciones=[posicion for posicion, char in enumerate(t) if char == '_']
                        if t[0]=='N'and t[posiciones[0]+1:posiciones[0+1]]==str(i) and t[posiciones[1]+1:]=='0':
                                restriccion_indices.append(data.diccionario_indices[t])
                                restriccion_valores.append(-1)
                        elif t[0]=='W'and t[posiciones[0]+1:posiciones[0+1]]==str(i) and t[posiciones[1]+1:]=='0':
                                restriccion_indices.append(data.diccionario_indices[t])
                                restriccion_valores.append(5)                                                
                indices = restriccion_indices
                values = restriccion_valores
                row = [indices,values]
                #print(row)
                if len(indices)>0:
                        my_problem.linear_constraints.add(lin_expr=[row], senses=['L'], rhs=[0])
                else: 
                        break  

In [18]:
def add_constraint_matrix_13(my_problem, data):
# SUM (N_i_n) <= 5 para todo i  para n=0    #X_{i}_{j}_{k}_{l}  ||  R_{j}_{k}_{l}   ||D_{i}_{k} || N_{i}_{n} || W_{i}_{w}
        for i in range(data.cantidad_trabajadores):
                restriccion_indices=[]
                restriccion_valores=[]
                for t in data.diccionario_indices.keys():
                        posiciones=[posicion for posicion, char in enumerate(t) if char == '_']
                        if t[0]=='N'and t[posiciones[0]+1:posiciones[0+1]]==str(i) and t[posiciones[1]+1:]=='0':
                                restriccion_indices.append(data.diccionario_indices[t])
                                restriccion_valores.append(1)
                                            
                indices = restriccion_indices
                values = restriccion_valores
                row = [indices,values]
                #print(row)
                if len(indices)>0:
                        my_problem.linear_constraints.add(lin_expr=[row], senses=['L'], rhs=[5])
                else: 
                        break  

In [19]:
def add_constraint_matrix_14(my_problem, data):
# 5 * W_i_1 - N_i_1 <= 0 para todo i  para     #X_{i}_{j}_{k}_{l}  ||  R_{j}_{k}_{l}   ||D_{i}_{k} || N_{i}_{n} || W_{i}_{w}
        for i in range(data.cantidad_trabajadores):
                restriccion_indices=[]
                restriccion_valores=[]
                for t in data.diccionario_indices.keys():
                        posiciones=[posicion for posicion, char in enumerate(t) if char == '_']
                        if t[0]=='N'and t[posiciones[0]+1:posiciones[0+1]]==str(i) and t[posiciones[1]+1:]=='1':
                                restriccion_indices.append(data.diccionario_indices[t])
                                restriccion_valores.append(-1)
                        elif t[0]=='W'and t[posiciones[0]+1:posiciones[0+1]]==str(i) and t[posiciones[1]+1:]=='1':
                                restriccion_indices.append(data.diccionario_indices[t])
                                restriccion_valores.append(5)                                                
                indices = restriccion_indices
                values = restriccion_valores
                row = [indices,values]
                #print(row)
                if len(indices)>0:
                        my_problem.linear_constraints.add(lin_expr=[row], senses=['L'], rhs=[0])
                else: 
                        break 

In [20]:
def add_constraint_matrix_15(my_problem, data):
#    -5 * W_i_0 + N_i_1 <= 0 para todo i  para     #X_{i}_{j}_{k}_{l}  ||  R_{j}_{k}_{l}   ||D_{i}_{k} || N_{i}_{n} || W_{i}_{w}
        for i in range(data.cantidad_trabajadores):
                restriccion_indices=[]
                restriccion_valores=[]
                for t in data.diccionario_indices.keys():
                        posiciones=[posicion for posicion, char in enumerate(t) if char == '_']
                        if t[0]=='N'and t[posiciones[0]+1:posiciones[0+1]]==str(i) and t[posiciones[1]+1:]=='1':
                                restriccion_indices.append(data.diccionario_indices[t])
                                restriccion_valores.append(1)
                        elif t[0]=='W'and t[posiciones[0]+1:posiciones[0+1]]==str(i) and t[posiciones[1]+1:]=='0':
                                restriccion_indices.append(data.diccionario_indices[t])
                                restriccion_valores.append(-5)                                                
                indices = restriccion_indices
                values = restriccion_valores
                row = [indices,values]
                #print(row)
                if len(indices)>0:
                        my_problem.linear_constraints.add(lin_expr=[row], senses=['L'], rhs=[0])
                else: 
                        break 

In [21]:
def add_constraint_matrix_16(my_problem, data):
# 5 * W_i_2 - N_i_2 <= 0 para todo i  para     #X_{i}_{j}_{k}_{l}  ||  R_{j}_{k}_{l}   ||D_{i}_{k} || N_{i}_{n} || W_{i}_{w}
        for i in range(data.cantidad_trabajadores):
                restriccion_indices=[]
                restriccion_valores=[]
                for t in data.diccionario_indices.keys():
                        posiciones=[posicion for posicion, char in enumerate(t) if char == '_']
                        if t[0]=='N'and t[posiciones[0]+1:posiciones[0+1]]==str(i) and t[posiciones[1]+1:]=='2':
                                restriccion_indices.append(data.diccionario_indices[t])
                                restriccion_valores.append(-1)
                        elif t[0]=='W'and t[posiciones[0]+1:posiciones[0+1]]==str(i) and t[posiciones[1]+1:]=='2':
                                restriccion_indices.append(data.diccionario_indices[t])
                                restriccion_valores.append(5)                                                
                indices = restriccion_indices
                values = restriccion_valores
                row = [indices,values]
                #print(row)
                if len(indices)>0:
                        my_problem.linear_constraints.add(lin_expr=[row], senses=['L'], rhs=[0])
                else: 
                        break 

In [22]:
def add_constraint_matrix_17(my_problem, data):
#    -5 * W_i_1 + N_i_2 <= 0 para todo i  para     #X_{i}_{j}_{k}_{l}  ||  R_{j}_{k}_{l}   ||D_{i}_{k} || N_{i}_{n} || W_{i}_{w}
        for i in range(data.cantidad_trabajadores):
                restriccion_indices=[]
                restriccion_valores=[]
                for t in data.diccionario_indices.keys():
                        posiciones=[posicion for posicion, char in enumerate(t) if char == '_']
                        if t[0]=='N'and t[posiciones[0]+1:posiciones[0+1]]==str(i) and t[posiciones[1]+1:]=='2':
                                restriccion_indices.append(data.diccionario_indices[t])
                                restriccion_valores.append(1)
                        elif t[0]=='W'and t[posiciones[0]+1:posiciones[0+1]]==str(i) and t[posiciones[1]+1:]=='1':
                                restriccion_indices.append(data.diccionario_indices[t])
                                restriccion_valores.append(-5)                                                
                indices = restriccion_indices
                values = restriccion_valores
                row = [indices,values]
                #print(row)
                if len(indices)>0:
                        my_problem.linear_constraints.add(lin_expr=[row], senses=['L'], rhs=[0])
                else: 
                        break 

In [23]:
def add_constraint_matrix_18(my_problem, data):
# N_i_n >= 0 para todo i  para n=3   #X_{i}_{j}_{k}_{l}  ||  R_{j}_{k}_{l}   ||D_{i}_{k} || N_{i}_{n} || W_{i}_{w}
        for i in range(data.cantidad_trabajadores):
                restriccion_indices=[]
                restriccion_valores=[]
                for t in data.diccionario_indices.keys():
                        posiciones=[posicion for posicion, char in enumerate(t) if char == '_']
                        if t[0]=='N'and t[posiciones[0]+1:posiciones[0+1]]==str(i) and t[posiciones[1]+1:]=='3':
                                restriccion_indices.append(data.diccionario_indices[t])
                                restriccion_valores.append(-1)
                                            
                indices = restriccion_indices
                values = restriccion_valores
                row = [indices,values]
                #print(row)
                if len(indices)>0:
                        my_problem.linear_constraints.add(lin_expr=[row], senses=['L'], rhs=[0])
                else: 
                        break  

In [24]:
def add_constraint_matrix_19(my_problem, data):
#    -15 * W_i_2 + N_i_3 <= 0 para todo i  para     #X_{i}_{j}_{k}_{l}  ||  R_{j}_{k}_{l}   ||D_{i}_{k} || N_{i}_{n} || W_{i}_{w}
        for i in range(data.cantidad_trabajadores):
                restriccion_indices=[]
                restriccion_valores=[]
                for t in data.diccionario_indices.keys():
                        posiciones=[posicion for posicion, char in enumerate(t) if char == '_']
                        if t[0]=='N'and t[posiciones[0]+1:posiciones[0+1]]==str(i) and t[posiciones[1]+1:]=='3':
                                restriccion_indices.append(data.diccionario_indices[t])
                                restriccion_valores.append(1)
                        elif t[0]=='W'and t[posiciones[0]+1:posiciones[0+1]]==str(i) and t[posiciones[1]+1:]=='2':
                                restriccion_indices.append(data.diccionario_indices[t])
                                restriccion_valores.append(-15)                                                
                indices = restriccion_indices
                values = restriccion_valores
                row = [indices,values]
                
                if len(indices)>0:
                        my_problem.linear_constraints.add(lin_expr=[row], senses=['L'], rhs=[0])
                else: 
                        break

In [25]:
def add_constraint_matrix_20A(my_problem, data):
#     #X_{i}_{j}_{k}_{l}  ||  R_{j}_{k}_{l}   ||D_{i}_{k} || N_{i}_{n} || W_{i}_{w}
        for i in range(data.cantidad_trabajadores):
                restriccion_indices=[]
                restriccion_valores=[]
                for t in data.diccionario_indices.keys():
                        posiciones=[posicion for posicion, char in enumerate(t) if char == '_']
                        if t[0]=='W'and t[posiciones[0]+1:posiciones[0+1]]==str(i) and t[posiciones[1]+1:]=='1':
                                restriccion_indices.append(data.diccionario_indices[t])
                                restriccion_valores.append(1)
                        elif t[0]=='W'and t[posiciones[0]+1:posiciones[0+1]]==str(i) and t[posiciones[1]+1:]=='0':
                                restriccion_indices.append(data.diccionario_indices[t])
                                restriccion_valores.append(-1)                                                
                indices = restriccion_indices
                values = restriccion_valores
                row = [indices,values]
                if len(indices)>0:
                        my_problem.linear_constraints.add(lin_expr=[row], senses=['L'], rhs=[0])
                else: 
                        break

In [26]:
def add_constraint_matrix_20B(my_problem, data):
#     #X_{i}_{j}_{k}_{l}  ||  R_{j}_{k}_{l}   ||D_{i}_{k} || N_{i}_{n} || W_{i}_{w}
        for i in range(data.cantidad_trabajadores):
                restriccion_indices=[]
                restriccion_valores=[]
                for t in data.diccionario_indices.keys():
                        posiciones=[posicion for posicion, char in enumerate(t) if char == '_']
                        if t[0]=='W'and t[posiciones[0]+1:posiciones[0+1]]==str(i) and t[posiciones[1]+1:]=='2':
                                restriccion_indices.append(data.diccionario_indices[t])
                                restriccion_valores.append(1)
                        elif t[0]=='W'and t[posiciones[0]+1:posiciones[0+1]]==str(i) and t[posiciones[1]+1:]=='1':
                                restriccion_indices.append(data.diccionario_indices[t])
                                restriccion_valores.append(-1)                                                
                indices = restriccion_indices
                values = restriccion_valores
                row = [indices,values]
                
                if len(indices)>0:
                        my_problem.linear_constraints.add(lin_expr=[row], senses=['L'], rhs=[0])
                else: 
                        break

In [27]:
#        I=self.cantidad_trabajadores  #trabajador
#        J=self.cantidad_ordenes  #orden o tarea
#        K=self.dias_semana  #dia de semana
#        L=self.turnos  #turno
#        N=self.tramos_salario  #tramos
#        W=N-1 
#data.cantidad_ordenes
#data.ordenes[9].trabajadores_necesarios

In [28]:
def populate_by_row(my_problem, data):
#VERdef add_constraint_matrix_4(my_problem, data):

    # Definimos y agregamos las variables de la parte positiva de la formula beneficio
        coef_fun_objetivo_1 = data.variables_binarias
    
        my_problem.variables.add(
                obj = coef_fun_objetivo_1, 
                lb =[0]*len(data.variables_binarias), 
                ub =[1]*len(data.variables_binarias), 
                types=['B']*len(data.variables_binarias)
                ) 

        # Definimos y agregamos las variables de la parte negativa de la formula costo
        #cada trabajador tiene N1, N2, N3, N4  si el indice que les genera es posterior al resto esta justo!!!
        coef_fun_objetivo_2=data.variables_N

        my_problem.variables.add(
                obj = coef_fun_objetivo_2, 
                lb =[0]*len(data.variables_N), 
                ub =[5]*len(data.variables_N), 
                types=['I']*len(data.variables_N)
                ) 

        # Seteamos direccion del problema
        my_problem.objective.set_sense(my_problem.objective.sense.maximize)
        # ~ my_problem.objective.set_sense(my_problem.objective.sense.minimize)

        # Definimos las restricciones del modelo. Encapsulamos esto en una funcion. 
        add_constraint_matrix_1(my_problem, data)
        add_constraint_matrix_2(my_problem, data)
        add_constraint_matrix_3(my_problem, data)
        add_constraint_matrix_4(my_problem, data)
        add_constraint_matrix_5(my_problem, data)
        add_constraint_matrix_6(my_problem, data)
        add_constraint_matrix_7A(my_problem, data)
        add_constraint_matrix_7B(my_problem, data)
        add_constraint_matrix_8(my_problem, data)
        add_constraint_matrix_9(my_problem, data)
        add_constraint_matrix_10(my_problem, data)
        add_constraint_matrix_11(my_problem, data)
        add_constraint_matrix_12(my_problem, data)
        add_constraint_matrix_13(my_problem, data)
        add_constraint_matrix_14(my_problem, data)
        add_constraint_matrix_15(my_problem, data)
        add_constraint_matrix_16(my_problem, data)
        add_constraint_matrix_17(my_problem, data)
        add_constraint_matrix_18(my_problem, data)
        add_constraint_matrix_19(my_problem, data)
        add_constraint_matrix_20A(my_problem, data)
        add_constraint_matrix_20B(my_problem, data)
        # Exportamos el LP cargado en myprob con formato .lp. - Util para debug.
        my_problem.write('balanced_assignment_P2_5.lp')

In [29]:

# Armamos el modelo.
populate_by_row(my_problem,data)

Default variable names x1, x2 ... being created.
Default row names c1, c2 ... being created.


In [30]:
def solve_lp(my_problem, data):
    
    # Resolvemos el ILP.
    
    my_problem.solve()

    # Obtenemos informacion de la solucion. Esto lo hacemos a traves de 'solution'. 
    x_variables = my_problem.solution.get_values()
    objective_value = my_problem.solution.get_objective_value()
    status = my_problem.solution.get_status()
    status_string = my_problem.solution.get_status_string(status_code = status)

    print('Funcion objetivo: ',objective_value)
    print('Status solucion: ',status_string,'(' + str(status) + ')')

    # Imprimimos las variables usadas.
    for i in range(len(x_variables)):
        # Tomamos esto como valor de tolerancia, por cuestiones numericas.
        if x_variables[i] > TOLERANCE:
            #print('x_' + str(list(data.diccionario_indices.keys())[i-1]) + ':' , x_variables[i])
            print(str(list(data.diccionario_indices.keys())[i]) + ':' , x_variables[i])
     
        #print(str(list(data.diccionario_indices.keys())[i-1]) + ':' , x_variables[i])

In [31]:
solve_lp(my_problem,data)



Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
Found incumbent of value 0.000000 after 0.00 sec. (0.04 ticks)
Tried aggregator 2 times.
MIP Presolve eliminated 976 rows and 240 columns.
MIP Presolve modified 278 coefficients.
Aggregator did 264 substitutions.
Reduced MIP has 490 rows, 602 columns, and 5536 nonzeros.
Reduced MIP has 594 binaries, 8 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (13.46 ticks)
Probing time = 0.00 sec. (3.60 ticks)
Tried aggregator 1 time.
Detecting symmetries...
MIP Presolve modified 207 coefficients.
Reduced MIP has 490 rows, 602 columns, and 5329 nonzeros.
Reduced MIP has 594 binaries, 8 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (7.52 ticks)
Probing time = 0.00 sec. (3.59 ticks)
Clique table members: 2602.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 8 threads.
Root relaxation solu

'''
def main():
    
    # Obtenemos los datos de la instancia.
    data = get_instance_data()
    
    genera_condiciones_variables()
    
    # Definimos el problema de cplex.
    my_problem = cplex.Cplex()
    
    # Armamos el modelo.
    populate_by_row(my_problem,data)

    # Resolvemos el modelo.
    solve_lp(my_problem,data)


#if __name__ == '__main__':
#    main()
